# Building and Visualizing molecules

This section describe to to model molecule using [ASE](https://wiki.fysik.dtu.dk/ase)

For visualization
- ASE provide a [Visualize module](https://wiki.fysik.dtu.dk/ase//ase/visualize/visualize.html). Consider to install packages `nglview` may require to visualize in jupyter.
- See [Nglview advance usage](https://osscar-docs.readthedocs.io/en/latest/visualizer/nglview_advanced.html)
  ```
  %pip install git+https://github.com/arose/nglview
  ```

## From scratch

When there is no data file for molecules, we define it by hand

In [2]:
# Load modules
from ase import Atoms, Atom
from ase.visualize import view
import numpy as np

In [5]:
# define an Atoms object
atoms = Atoms([Atom('C', [0., 0., 0.]),
                Atom('O', [1.1, 0., 0.])],
                cell=(10, 10, 10))

print('V = {0:1.0f} Ang^3'.format(atoms.get_volume()))     

# visualize
atoms.pbc = True     # to view box
view(atoms, viewer='ngl')

V = 1000 Ang^3


There are two **inconvenient features** of the simple cubic cell:
1. Since the CO molecule is at the corner, its electron density is spread over the 8 corners of the box, which is not convenient for visualization later (see Visualizing electron density).
2. Due to the geometry of the cube, you need fairly large cubes to make sure the electron density of the molecule does not overlap with that of its images. Electron-electron interactions are repulsive, and the overlap makes the energy *increase* significantly.

The first problem is easily solved by centering the atoms in the unit cell. The second problem can be solved by using a face-centered cubic lattice, which is the lattice with the closest packing. We show the results of the centering below, where we have guessed values for `b` until the CO molecules are on average 10 Å apart. 

Note the final volume is only about 715 $\AA^3$, which is smaller than the cube. This will result in *less computational time* to compute properties.

In [6]:
## use FCC lattice
b = 7.1
atoms = Atoms([Atom('C', [0., 0., 0.]),
                Atom('O', [1.1, 0., 0.])],
                cell=[[b, b, 0.],
                [b, 0., b],
                [0., b, b]])
atoms.center() # translate atoms to center of unit cell

atoms.pbc = True
view(atoms, viewer='ngl')

**How do you know the distance to the neighboring image?**

All we have to do is figure out the length of each lattice vector, because these are what separate the atoms in the images.

In [46]:
# get unit cell vectors and their lengths
(a1, a2, a3) = atoms.get_cell()
print('|a1| = {0:1.2f} Ang'.format(np.sum(a1**2)**0.5))
print('|a2| = {0:1.2f} Ang'.format(np.linalg.norm(a2)))
print('|a3| = {0:1.2f} Ang'.format(np.sum(a3**2)**0.5))

print('V = {0:1.0f} Ang^3'.format(atoms.get_volume()))    

|a1| = 10.04 Ang
|a2| = 10.04 Ang
|a3| = 10.04 Ang
V = 716 Ang^3


## Reading other data formats

[`ase.io`](https://wiki.fysik.dtu.dk/ase/ase/io/io.html) supports many different file formats.

A covenient way if reading `.cif` format from [Crystallography Open Database](http://www.crystallography.net/cod/) from [Material Project](https://materialsproject.org/)

```note 
The XYZ format does not have unit cell information in it, so you will have to figure out a way to provide it.
`atoms.center(vacuum=5)` in `ase` is a qick for that
```

In [8]:
from ase.io import read, write

atoms = read('molecules/isobutane.xyz')

atoms.center(vacuum=3)
atoms.pbc = True
view(atoms, viewer='ngl')

## ASE's Predefined molecules

[`ase.data`]() contains a broad set of atoms
and molecules for which good experimental data exists, making them useful for benchmarking studies

In [52]:
from ase.data import g2
keys = g2.data.keys()
keys

dict_keys(['Be', 'BeH', 'C', 'C2H2', 'C2H4', 'C2H6', 'CH', 'CH2_s1A1d', 'CH2_s3B1d', 'CH3', 'CH3Cl', 'CH3OH', 'CH3SH', 'CH4', 'CN', 'CO', 'CO2', 'CS', 'Cl', 'Cl2', 'ClF', 'ClO', 'F', 'F2', 'H', 'H2CO', 'H2O', 'H2O2', 'HCN', 'HCO', 'HCl', 'HF', 'HOCl', 'Li', 'Li2', 'LiF', 'LiH', 'N', 'N2', 'N2H4', 'NH', 'NH2', 'NH3', 'NO', 'Na', 'Na2', 'NaCl', 'O', 'O2', 'OH', 'P', 'P2', 'PH2', 'PH3', 'S', 'S2', 'SH2', 'SO', 'SO2', 'Si', 'Si2', 'Si2H6', 'SiH2_s1A1d', 'SiH2_s3B1d', 'SiH3', 'SiH4', 'SiO', '2-butyne', 'Al', 'AlCl3', 'AlF3', 'B', 'BCl3', 'BF3', 'C2Cl4', 'C2F4', 'C2H3', 'C2H5', 'C2H6CHOH', 'C2H6NH', 'C2H6SO', 'C3H4_C2v', 'C3H4_C3v', 'C3H4_D2d', 'C3H6_Cs', 'C3H6_D3h', 'C3H7', 'C3H7Cl', 'C3H8', 'C3H9C', 'C3H9N', 'C4H4NH', 'C4H4O', 'C4H4S', 'C5H5N', 'C5H8', 'C6H6', 'CCH', 'CCl4', 'CF3CN', 'CF4', 'CH2NHCH2', 'CH2OCH2', 'CH2SCH2', 'CH3CH2Cl', 'CH3CH2NH2', 'CH3CH2O', 'CH3CH2OCH3', 'CH3CH2OH', 'CH3CH2SH', 'CH3CHO', 'CH3CN', 'CH3CO', 'CH3COCH3', 'CH3COCl', 'CH3COF', 'CH3CONH2', 'CH3COOH', 'CH3NO2', 

In [14]:
from ase.build import molecule

atoms = molecule('CH3CN')

atoms.center(vacuum=3)
atoms.pbc = True
view(atoms, viewer='ngl')

## Manipulate molecules

[`ase.Atoms`](https://wiki.fysik.dtu.dk/ase/gettingstarted/manipulating_atoms/manipulating_atoms.html) provides many ways for manipulates molecules

### Rotate molecules

Using `ase.Atoms.rotate`, and consider:
- Whether the center of mass should be fixed or not
- Whether or not the unit cell should be rotated

In [15]:
from ase.build import molecule

atoms = molecule('CH3CN')
atoms.center(vacuum=6)
atoms.pbc = True
p1 = atoms.get_positions()

atoms.rotate('x', np.pi/4, center='COM', rotate_cell=False)
atoms.rotate('y', np.pi/4, center='COM', rotate_cell=False)

print('difference in positions after rotating')
print('atom    difference vector')
print('--------------------------------------')
p2 = atoms.get_positions()
diff = p2 - p1
for i, d in enumerate(diff):
    print('{0} {1}'.format(i, d))

# visualize
view(atoms, viewer='ngl')

difference in positions after rotating
atom    difference vector
--------------------------------------
0 [-0.01782051  0.01782219  0.0002443 ]
1 [ 2.20136479e-03 -2.20157163e-03 -3.01777230e-05]
2 [ 0.01835166 -0.01835339 -0.00025158]
3 [-0.02277373  0.02287218  0.01436336]
4 [-0.02314601  0.02301662 -0.01887695]
5 [-0.02297922  0.02301662  0.0054581 ]


### Combine Atoms objects


In [58]:
from ase.build import molecule

atoms1 = molecule('NH3')
atoms2 = molecule('O2')
atoms2.translate([3, 0, 0])

atoms = atoms1 + atoms2

# atoms.pbc = True
view(atoms, viewer='ngl')